In [ ]:
import argparse
import multiprocessing
import os
import time
from itertools import product

import numpy as np
import yaml

from config import SPECS_REDUCED_ACTIVS_2000_YAML, SPECS_FLOOD_YAMLS, HEURISTIC_RESULTS_DIR
from util import Grid, ResilienceState, in_notebook

In [ ]:
# user inputs
if in_notebook():
    f_min = 0
    f_max = 20
    f_inc = 1
    r_hat = 3
    casestudy = 'imelda'
    approach = 'stochastic'
    num_processes = 8
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--fmin')
    parser.add_argument('--fmax')
    parser.add_argument('--finc')
    parser.add_argument('--rhat')
    parser.add_argument('--casestudy')
    parser.add_argument('--pftype')
    parser.add_argument('--approach')
    parser.add_argument('--numprocesses')
    args = parser.parse_args()
    f_min = int(args.fmin)
    f_max = int(args.fmax)
    f_inc = int(args.finc)
    r_hat = int(args.rhat)
    pftype = str(args.pftype)
    casestudy = str(args.casestudy)
    approach = str(args.approach)
    num_processes = int(args.numprocesses)

In [ ]:
# load data
specs = {}
with open(SPECS_REDUCED_ACTIVS_2000_YAML) as fh:
    specs.update(yaml.load(fh, Loader=yaml.Loader))
with open(SPECS_FLOOD_YAMLS[casestudy,]) as fh:
    specs.update(yaml.load(fh, Loader=yaml.Loader))
    for key, val in specs['r_hat'].items():
        specs['r_hat'][key] = min(val, r_hat)
    for key in list(specs['xi']):
        (k, r, omega) = key
        if r > r_hat:
            specs['xi'].pop(key)
    for k in specs['R']:
        specs['R'][k] = [i for i in range(1, min(max(specs['R'][k]), r_hat) + 1)]

In [ ]:
# construct the grid and resilience state
grid = [Grid(specs)]

budget = list(np.arange(f_min, f_max + f_inc, f_inc))
degree = [1]
weight_load = [1.000]
weight_flow = [0.000, 0.025, 0.050, 0.075, 0.100, 0.125, 0.150]
overspend = [False]

args = product(grid, budget, degree, weight_load, weight_flow, overspend)


def get_greedy_solution(grid, budget, degree, weight_load, weight_flow, overspend):
    t0 = time.time()
    rstate = ResilienceState(grid)
    if approach == 'stochastic':
        sol = rstate.adopt_stochastic_greedy_solution(budget, degree, weight_load, weight_flow, overspend)
    elif approach == 'robust':
        sol = rstate.adopt_robust_greedy_solution(budget, degree, weight_load, weight_flow, overspend)
    else:
        raise ValueError(approach)
    tf = time.time()
    print(round(tf - t0, 4), budget, weight_load, weight_flow, flush=True)
    return (budget, degree, weight_load, weight_flow, overspend), sol


with multiprocessing.Pool(num_processes) as pool:
    results = pool.starmap(get_greedy_solution, args)

solutions = dict()
for (budget, _, weight_load, weight_flow, _), sol in results:
    solutions[int(budget), float(weight_load), float(weight_flow)] = sol

os.makedirs(os.path.join(HEURISTIC_RESULTS_DIR, approach), exist_ok=True)
heur_sols_filename = os.path.join(HEURISTIC_RESULTS_DIR, approach, f'solutions-{casestudy}-r{r_hat}.yaml')
with open(heur_sols_filename, 'w') as fh:
    yaml.dump(solutions, fh)